# Section 2: Data Wrangling

In [2]:
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
energy=pd.read_csv("../data/raw/energy_dataset.csv", parse_dates=['time'], index_col='time')
weather=pd.read_csv("../data/raw/weather_features.csv", parse_dates=['dt_iso'], index_col=['dt_iso'])

In [4]:
energy.index=pd.to_datetime(energy.index, utc=True)+pd.DateOffset(hours=1)
weather.index=pd.to_datetime(weather.index, utc=True)+pd.DateOffset(hours=1)

# 1. Energy Dataset

In [5]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2015-01-01 00:00:00+00:00 to 2018-12-31 23:00:00+00:00
Freq: H
Data columns (total 28 columns):
generation biomass                             35045 non-null float64
generation fossil brown coal/lignite           35046 non-null float64
generation fossil coal-derived gas             35046 non-null float64
generation fossil gas                          35046 non-null float64
generation fossil hard coal                    35046 non-null float64
generation fossil oil                          35045 non-null float64
generation fossil oil shale                    35046 non-null float64
generation fossil peat                         35046 non-null float64
generation geothermal                          35046 non-null float64
generation hydro pumped storage aggregated     0 non-null float64
generation hydro pumped storage consumption    35045 non-null float64
generation hydro run-of-river and poundage     35045 non-null float64


For further analysis and modelling the following points are considered:
1. Energy generation data is not needed as demand is being modelled.
2. 'total load actual' is total energy demand, the variable being attempted to predict in this project.
2. 'total load forecast' is the demand forecasted by the TSO(Transmission Service Operator) in Spain.

But it looks like 'total load actual' is less than the forecast, let's examine and replace the missing values.



In [15]:
energy_new=energy[['total load actual','total load forecast']]

In [16]:
energy_new[energy_new['total load actual'].isnull()]

,total load actual,total load forecast
time,,
2015-01-05 12:00:00+00:00,NaN,23209.0
2015-01-05 13:00:00+00:00,NaN,23725.0
2015-01-05 14:00:00+00:00,NaN,23614.0
2015-01-05 15:00:00+00:00,NaN,22381.0
2015-01-05 16:00:00+00:00,NaN,21371.0
2015-01-05 17:00:00+00:00,NaN,20760.0
2015-01-28 13:00:00+00:00,NaN,36239.0
2015-02-01 07:00:00+00:00,NaN,24379.0
2015-02-01 08:00:00+00:00,NaN,27389.0


In [14]:
(energy_new['total load actual'].isnull()).sum()

36

There is missing data for 36 hours. Some are consecutive 10 hours. If we choose to drop these values, a full 50% of day of year's data is gone. It is best to replace them.

In order to replace them with accurate values, the following must be considered:

1. The hour of day
2. The day of the week
3. The month of the year

A good value to account for the above 3 aboce points would be to replace missing values with average of 6 values that represent:

1. 3 weeks of future values and 3 weeks of past values.
2. On the same day of week
3. On the same hour of day

In [61]:
indicies_nan=energy_new[energy_new['total load actual'].isnull()].index
for idx in indicies_nan:
    past_future_3w= energy_new[(energy_new.index>(idx-pd.to_timedelta(3, unit='w'))) & (energy_new.index>(idx+pd.to_timedelta(3, unit='w')))]['total load actual']
    dow=idx.dayofweek
    hod=idx.hour
    past_future_3w= past_future_3w[past_future_3w.index.dayofweek==dow]
    past_future_3w= past_future_3w[past_future_3w.index.hour==hod]
    energy_new['total load actual'].loc[idx]= past_future_3w.mean()

C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yfawz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [64]:
energy_new.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2015-01-01 00:00:00+00:00 to 2018-12-31 23:00:00+00:00
Freq: H
Data columns (total 2 columns):
total load actual      35064 non-null float64
total load forecast    35064 non-null float64
dtypes: float64(2)
memory usage: 2.1 MB


In [77]:
energy_new.head()

,total load actual,total load forecast
time,,
2015-01-01 00:00:00+00:00,25385.0,26118.0
2015-01-01 01:00:00+00:00,24382.0,24934.0
2015-01-01 02:00:00+00:00,22734.0,23515.0
2015-01-01 03:00:00+00:00,21286.0,22642.0
2015-01-01 04:00:00+00:00,20264.0,21785.0


In [78]:
energy_new.to_csv('C:\\Users\\yfawz\\OneDrive\\Desktop\\energy_demand_forecasting\\data\\processed\\load_data.csv')

# 2. Weather Data

From our initial EDA, we have seen that there were some weather features that had outliers. Let's examine and deal with those while converting the temperature from Kelvin to Celsius.

In [81]:
weather.describe()

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id
count,178396.000000,178396.000000,178396.000000,1.783960e+05,178396.000000,178396.00000,178396.000000,178396.000000,178396.000000,178396.000000,178396.000000,178396.000000
mean,289.618605,288.330442,291.091267,1.069261e+03,68.423457,2.47056,166.591190,0.075492,0.000380,0.004763,25.073292,759.831902
std,8.026199,7.955491,8.612454,5.969632e+03,21.902888,2.09591,116.611927,0.398847,0.007288,0.222604,30.774129,108.733223
min,262.240000,262.240000,262.240000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,200.000000
25%,283.670000,282.483602,284.650000,1.013000e+03,53.000000,1.00000,55.000000,0.000000,0.000000,0.000000,0.000000,800.000000
50%,289.150000,288.150000,290.150000,1.018000e+03,72.000000,2.00000,177.000000,0.000000,0.000000,0.000000,20.000000,800.000000
75%,295.150000,293.730125,297.150000,1.022000e+03,87.000000,4.00000,270.000000,0.000000,0.000000,0.000000,40.000000,801.000000
max,315.600000,315.150000,321.150000,1.008371e+06,100.000000,133.00000,360.000000,12.000000,2.315000,21.500000,100.000000,804.000000


## The following observations need to be removed examined:

1. The temperature shall be converted to Celsius as more relatable to get an insight from celsius.


2. The pressure seems to have abnormally high maximum values which seem impossible. This is a data error. These values are in Hecto Pascal. 1000 hPa is standard atmospheric pressure in the air. 10,000 HectoPascal is equivalent to a Sedan car being supported on the palm of an average human hand. This is enough to kill a person. Imagine 1,000,000 HectoPascal. The highest and lowest pressure recorded on earth is 1084hPa and 870hPa respectively.


- We shall replace values above 1080 and below 870 these results with the mean of that date where values are less than 1080 hPa.


3. The maximum windspeed recorded seems unlikely to have occured given that the fastest wind speed ever recorded on earth is 103 m/s and a category 5 hurricane is 70m/s. Any wind speed above 60 will be replaces with the mean of dayofyear.

In [82]:
weather=weather.groupby(weather.index).mean()
weather_new=weather

In [83]:
#adding temperature in Celsius
weather_new["temp_c"]=weather_new.temp-273.15
weather_new["temp_c_max"]=weather_new.temp_max-273.15
weather_new["temp_c_min"]=weather_new.temp_min-273.15

In [84]:
#replacing wind speed errors
indicies_ws=weather_new[weather_new.wind_speed>70].wind_speed.index #finding indicies where wind speed is extremely high
for idx in indicies_ws:
    weather_new.wind_speed.loc[idx]=weather[weather.index.month==weather.loc[idx].name.month].mean().wind_speed

In [86]:
#replacing pressure errors
indicies_p=weather_new[(weather_new.pressure<870) | (weather_new.pressure>1080)].pressure.index
for idx in indicies_p:
    day_press=weather[weather.index.date==pd.datetime.date(weather.loc['2015-02-20 09:00:00+00:00'].name)].pressure
    weather_new.pressure.loc[idx]=weather.pressure[weather.pressure<1080].mean()

In [87]:
weather_new.columns

Index(['temp', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h', 'clouds_all', 'weather_id',
       'temp_c', 'temp_c_max', 'temp_c_min'],
      dtype='object')

The following columns will be needed:

1. All 'temp_c' data
2. 'pressure'
3. 'wind_speed'

The following columns will not be need:

1. All 'temp' data as we have the newly made 'temp_c'
2. 'weather_id'
3. 'wind_deg' as this only affect wind power generation.
4. All 'rain' and 'snow' data as this only affects hydro power generation.

In [88]:
weather_final=weather_new[['temp_c','temp_c_max','temp_c_min','pressure','humidity','wind_speed']]

In [89]:
weather_final.describe()

,temp_c,temp_c_max,temp_c_min,pressure,humidity,wind_speed
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000
mean,16.530183,17.997454,15.246642,1016.242160,68.217013,2.467529
std,7.261069,7.507959,7.121843,8.287243,14.890167,1.357751
min,-1.208138,-1.208138,-2.600000,933.200000,22.600000,0.000000
25%,10.910000,12.200000,9.800000,1012.800000,56.800000,1.400000
50%,15.875000,17.400000,14.600000,1016.857143,69.800000,2.200000
75%,21.936000,23.600000,20.460000,1021.000000,80.200000,3.200000
max,36.116000,40.398000,35.400000,1039.800000,100.000000,29.800000


In [90]:
weather_final.head()

,temp_c,temp_c_max,temp_c_min,pressure,humidity,wind_speed
dt_iso,,,,,,
2015-01-01 00:00:00+00:00,-0.658537,-0.658537,-0.658537,1016.4,82.4,2.0
2015-01-01 01:00:00+00:00,-0.637300,-0.637300,-0.637300,1016.2,82.4,2.0
2015-01-01 02:00:00+00:00,-1.050862,-1.050862,-1.050862,1016.8,82.0,2.4
2015-01-01 03:00:00+00:00,-1.060531,-1.060531,-1.060531,1016.6,82.0,2.4
2015-01-01 04:00:00+00:00,-1.004100,-1.004100,-1.004100,1016.6,82.0,2.4


In [91]:
weather_final.to_csv('C:\\Users\\yfawz\\OneDrive\\Desktop\\energy_demand_forecasting\\data\\processed\\weather_new_data.csv')